## Project Architecture


In [ ]:
!git clone https://github.com/ultralytics/ultralytics.git  # Clone YOLOv8 repo
!cd ultralytics

!pip install ultralytics
!pip install torch torchvision numpy opencv-python matplotlib grad-cam ultralytics


1. Input image
https://drive.google.com/file/d/1HFFAzC9S3yl8BtZ0WZGwt5h_HuK8OHdx/view?usp=sharing

2. Output image
https://drive.google.com/file/d/1fYcPXh60cLzIE1CLh0k6yNe89TAR7Ja6/view?usp=sharing

## Implementation

In [ ]:
import torch
from ultralytics import YOLO
from IPython.display import clear_output

# Check PyTorch and CUDA
device = "CUDA" if torch.cuda.is_available() else "CPU"
print(f"Setup complete. Using torch {torch.__version__} on {device}")

# Load a YOLOv8 model to test
model = YOLO("yolov8m.pt")  # Load the nano model
print("YOLOv8 model loaded successfully!")


In [ ]:
# Install Roboflow
!pip install roboflow

# Import required libraries
from roboflow import Roboflow

# Initialize Roboflow with API key
rf = Roboflow(api_key="iMVOMaxCVf9Q6wQNbSnb")

# Load the original house_alloc project
project_house = rf.workspace("quantela").project("house_alloc")
version_house = project_house.version(17)
dataset_house = version_house.download("yolov8")
print("House allocation dataset downloaded successfully!")

# Load the new tree-seg project
project_tree = rf.workspace("test-4udyq").project("tree-seg")
version_tree = project_tree.version(1)  # Assuming version 1, adjust if different
dataset_tree = version_tree.download("yolov8")
print("Tree segmentation dataset downloaded successfully!")

In [ ]:
import os

# Define base paths for both datasets
base_path_house = '/content/house_alloc-17'  # Adjust to match your house dataset
base_path_tree = '/content/tree-seg-1'      # Adjust to match your tree dataset

# Define all required subdirectories
subdirs = [
    'train/images',
    'train/labels',
    'valid/images',
    'valid/labels',
    'test/images',
    'test/labels'
]

# Create directories for house dataset
try:
    for subdir in subdirs:
        dir_path = os.path.join(base_path_house, subdir)
        os.makedirs(dir_path, exist_ok=True)
    print("Directories for house YOLOv8 dataset created successfully!")
except Exception as e:
    print(f"Failed to create house directories: {e}")

# Create directories for tree dataset
try:
    for subdir in subdirs:
        dir_path = os.path.join(base_path_tree, subdir)
        os.makedirs(dir_path, exist_ok=True)
    print("Directories for tree YOLOv8 dataset created successfully!")
except Exception as e:
    print(f"Failed to create tree directories: {e}")

# Verify the structure
print("Created directory structure for house dataset:")
for subdir in subdirs:
    print(f"- {os.path.join(base_path_house, subdir)}")

print("Created directory structure for tree dataset:")
for subdir in subdirs:
    print(f"- {os.path.join(base_path_tree, subdir)}")

In [ ]:
# base_path = '/content/house_alloc-17'  # Match your Roboflow download

-------------------------------------------------------------


In [ ]:
# # Fix locale encoding
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# # Your original code
# dataset_location = dataset.location
# %cat {dataset_location}/data.yaml

# # This is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
# dataset_location = dataset.location
# %cat {dataset_location}/data.yaml

In [ ]:
import yaml
import os

# Fix locale encoding
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Dataset locations
dataset_location_house = dataset_house.location  # e.g., '/content/house_alloc-17'
dataset_location_tree = dataset_tree.location    # e.g., '/content/tree-seg-1'

# YAML paths
yaml_path_house = os.path.join(dataset_location_house, 'data.yaml').replace("\\", "/")
yaml_path_tree = os.path.join(dataset_location_tree, 'data.yaml').replace("\\", "/")

# Update house YAML
with open(yaml_path_house, 'r') as f:
    data_house = yaml.safe_load(f)
data_house['train'] = 'train/images'
data_house['val'] = 'valid/images'
data_house['test'] = 'test/images'
with open(yaml_path_house, 'w') as f:
    yaml.safe_dump(data_house, f)
print("House YAML updated!")

# Update tree YAML
with open(yaml_path_tree, 'r') as f:
    data_tree = yaml.safe_load(f)
data_tree['train'] = 'train/images'
data_tree['val'] = 'valid/images'
data_tree['test'] = 'test/images'
with open(yaml_path_tree, 'w') as f:
    yaml.safe_dump(data_tree, f)
print("Tree YAML updated!")

# Verify
print("House YAML:")
!cat {yaml_path_house}
print("Tree YAML:")
!cat {yaml_path_tree}

In [ ]:
!ls /content/ultralytics/ultralytics/cfg/models/v8/

In [ ]:
# This is the model configuration we will use for our tutorial
%cat /content/ultralytics/ultralytics/cfg/models/v8/yolov8-seg.yaml

In [ ]:
# Register the custom writetemplate magic
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

# Set number of classes
num_classes_house = 1  # For 'Houses' from house_alloc-17
num_classes_tree = 1   # Adjust based on tree-seg dataset (check data.yaml for 'names')

# House YOLOv8m custom config
config_text_house = f"""# YOLOv8 medium custom config for house detection
nc: {num_classes_house}  # Number of classes (1 for Houses)
depth_multiple: 0.67  # Model depth multiple (medium scale)
width_multiple: 0.75  # Layer channel multiple (medium scale)

# Backbone
backbone:
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]   # 2
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]   # 4
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]   # 6
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, True]]  # 8
  - [-1, 1, SPPF, [1024, 5]]    # 9 (SPP block)

# Head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 10
  - [[-1, 6], 1, Concat, [1]]  # 11 cat backbone P4
  - [-1, 3, C2f, [512, True]]   # 12
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 13
  - [[-1, 4], 1, Concat, [1]]  # 14 cat backbone P3
  - [-1, 3, C2f, [256, True]]   # 15 (P3/8-small)
  - [-1, 1, Conv, [256, 3, 2]]  # 16
  - [[-1, 12], 1, Concat, [1]]  # 17 cat head P4
  - [-1, 3, C2f, [512, True]]   # 18 (P4/16-medium)
  - [-1, 1, Conv, [512, 3, 2]]  # 19
  - [[-1, 9], 1, Concat, [1]]  # 20 cat head P5
  - [-1, 3, C2f, [1024, True]]  # 21 (P5/32-large)
  - [[15, 18, 21], 1, Detect, [nc]]  # Detect(P3, P4, P5)
"""

# Tree YOLOv8m segmentation custom config
config_text_tree = f"""# YOLOv8 medium custom config for tree segmentation
nc: {num_classes_tree}  # Number of classes (e.g., 1 for Trees)
depth_multiple: 0.67  # Model depth multiple (medium scale)
width_multiple: 0.75  # Layer channel multiple (medium scale)

# Backbone (same as detection)
backbone:
  - [-1, 1, Conv, [64, 3, 2]]   # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]   # 2
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]   # 4
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]   # 6
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, True]]  # 8
  - [-1, 1, SPPF, [1024, 5]]    # 9 (SPP block)

# Head (modified for segmentation)
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 10
  - [[-1, 6], 1, Concat, [1]]  # 11 cat backbone P4
  - [-1, 3, C2f, [512, True]]   # 12
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 13
  - [[-1, 4], 1, Concat, [1]]  # 14 cat backbone P3
  - [-1, 3, C2f, [256, True]]   # 15 (P3/8-small)
  - [-1, 1, Conv, [256, 3, 2]]  # 16
  - [[-1, 12], 1, Concat, [1]]  # 17 cat head P4
  - [-1, 3, C2f, [512, True]]   # 18 (P4/16-medium)
  - [-1, 1, Conv, [512, 3, 2]]  # 19
  - [[-1, 9], 1, Concat, [1]]   # 20 cat head P5
  - [-1, 3, C2f, [1024, True]]  # 21 (P5/32-large)
  - [[15, 18, 21], 1, Segment, [nc, 32, 256]]  # Segment(P3, P4, P5) - Segmentation head
"""

# Save the config files
config_path_house = "/content/yolov8m-custom-house.yaml"
with open(config_path_house, "w") as f:
    f.write(config_text_house)
print(f"House config file saved at {config_path_house}")

config_path_tree = "/content/yolov8m-custom-tree.yaml"
with open(config_path_tree, "w") as f:
    f.write(config_text_tree)
print(f"Tree config file saved at {config_path_tree}")

In [ ]:
# Update the Ultralytics repo for YOLOv8
!cd /content/ultralytics && git pull

print(dataset_location_house)
print(dataset_location_tree)

import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


In [ ]:
%%time
# Train house model
from ultralytics import YOLO

# Load pretrained YOLOv8m model for house detection
model_house = YOLO("yolov8m-custom-house.yaml").load("yolov8m.pt")  # Load with pre-trained weights

# Train the house model
model_house.train(
    data=f"{dataset_location_house}/data.yaml",  # Path to house dataset config
    epochs=20,                            # Reduced for demo; increase for better results
    imgsz=640,                           # Larger image size for better detection
    batch=16,                            # Batch size suitable for Tesla T4 GPU
    patience=20,                         # Early stopping after 5 epochs of no improvement
    cache="disk",                        # Use disk caching for deterministic results
    device=0,                            # Use GPU (Tesla T4, CUDA:0)
    workers=4,                           # Number of data loader workers
    pretrained=True,                     # Explicitly use pretrained weights
    lr0=0.0005,                          # Lower initial learning rate for stability
    mosaic=1.0,
    augment=True,                        # Enable data augmentation
    name="yolov8m_house_results",
)

# Train tree model
model_tree = YOLO("yolov8m-custom-tree.yaml").load("yolov8m-seg.pt")  # Load with pre-trained weights

# Train the tree model
model_tree.train(
    data=f"{dataset_location_tree}/data.yaml",  # Path to tree dataset config
    epochs=20 ,                            # Reduced for demo; increase for better results
    imgsz=640,                           # Larger image size for better detection
    batch=16,                            # Batch size suitable for Tesla T4 GPU
    patience=5,                         # Early stopping after 5 epochs of no improvement
    cache="disk",                        # Use disk caching for deterministic results
    device=0,                            # Use GPU (Tesla T4, CUDA:0)
    workers=4,                           # Number of data loader workers
    pretrained=True,                     # Explicitly use pretrained weights
    lr0=0.0005,                          # Lower initial learning rate for stability
    mosaic=1.0,
    augment=True,                        # Enable data augmentation
    name="yolov8m_tree_results",
)

# Optional: Validate both models after training
model_house.val()  # Runs validation on the best saved weights for house
model_tree.val()   # Runs validation on the best saved weights for tree

In [ ]:
!pip install huggingface_hub

In [ ]:
from google.colab import files
uploaded = files.upload()

img_path = list(uploaded.keys())[0]  # Get the uploaded file name
print("Using file:", img_path)

In [ ]:
from huggingface_hub import hf_hub_download
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2

# Load the input image using PIL and convert to RGB
image = Image.open(img_path).convert("RGB")
image_np = np.array(image)  # Convert to numpy array for processing

# Load trained YOLO models
model_house = YOLO("runs/detect/yolov8m_house_results/weights/best.pt")  # House detection model
# model_tree = YOLO("runs/detect/yolov8m_tree_results/weights/best.pt")  # Tree segmentation model

# Download and load the building segmentation model from Hugging Face
model_path = hf_hub_download(repo_id="keremberke/yolov8m-building-segmentation", filename="best.pt")
model_building = YOLO(model_path)  # Building segmentation model

# Run inference for all three models
results_house = model_house(image_np)
# results_tree = model_tree(image_np)
results_building = model_building(image_np)

# Define output directory inside Colab
output_dir = "/content/output"
os.makedirs(output_dir, exist_ok=True)  # Create folder if it doesn’t exist

# Extract original filename without extension
base_name = os.path.splitext(os.path.basename(img_path))[0]
output_path_combined = os.path.join(output_dir, f"{base_name}_combined_output.jpg")

# Create a copy of the original image for combined output
combined_image = image_np.copy()

# Process house detection results (bounding boxes)
if results_house and results_house[0].boxes is not None:  # Check if there are detections
    for r in results_house:
        boxes = r.boxes.xyxy.cpu().numpy()  # [x_min, y_min, x_max, y_max]
        confidences = r.boxes.conf.cpu().numpy()
        class_ids = r.boxes.cls.cpu().numpy()

        for box, conf, cls_id in zip(boxes, confidences, class_ids):
            x_min, y_min, x_max, y_max = map(int, box)
            # Draw house bounding box in blue
            cv2.rectangle(combined_image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)  # Blue color
            label = f"House: {conf:.2f}"
            cv2.putText(combined_image, label, (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# # Process tree segmentation results (masks)
# if results_tree and results_tree[0].masks is not None:  # Check if there are segmentation masks
#     for r in results_tree:
#         masks = r.masks.data.cpu().numpy()  # Segmentation masks
#         boxes = r.boxes.xyxy.cpu().numpy() if r.boxes is not None else None  # Bounding boxes (optional)
#         confidences = r.boxes.conf.cpu().numpy() if r.boxes is not None else None

#         for i, mask in enumerate(masks):
#             # Resize mask to match the input image dimensions
#             mask_resized = cv2.resize(mask, (combined_image.shape[1], combined_image.shape[0]),
#                                      interpolation=cv2.INTER_NEAREST)
#             mask_binary = (mask_resized > 0).astype(np.uint8) * 255

#             # Create a green overlay for the tree mask
#             colored_mask = np.zeros_like(combined_image)
#             colored_mask[:, :, 1] = mask_binary  # Green channel
#             combined_image = cv2.addWeighted(combined_image, 0.8, colored_mask, 0.5, 0)

#             # Optionally, add a label using the bounding box (if available)
#             if boxes is not None and confidences is not None and i < len(boxes):
#                 x_min, y_min, x_max, y_max = map(int, boxes[i])
#                 label = f"Tree: {confidences[i]:.2f}"
#                 cv2.putText(combined_image, label, (x_min, y_min - 10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Process building segmentation results (masks)
if results_building and results_building[0].masks is not None:  # Check if there are segmentation masks
    for r in results_building:
        masks = r.masks.data.cpu().numpy()  # Segmentation masks
        boxes = r.boxes.xyxy.cpu().numpy() if r.boxes is not None else None  # Bounding boxes (optional)
        confidences = r.boxes.conf.cpu().numpy() if r.boxes is not None else None

        for i, mask in enumerate(masks):
            # Resize mask to match the input image dimensions
            mask_resized = cv2.resize(mask, (combined_image.shape[1], combined_image.shape[0]),
                                     interpolation=cv2.INTER_NEAREST)
            mask_binary = (mask_resized > 0).astype(np.uint8) * 255

            # Create a red overlay for the building mask
            colored_mask = np.zeros_like(combined_image)
            colored_mask[:, :, 2] = mask_binary  # Red channel
            combined_image = cv2.addWeighted(combined_image, 0.8, colored_mask, 0.5, 0)

            # Optionally, add a label using the bounding box (if available)
            if boxes is not None and confidences is not None and i < len(boxes):
                x_min, y_min, x_max, y_max = map(int, boxes[i])
                label = f"Building: {confidences[i]:.2f}"
                cv2.putText(combined_image, label, (x_min, y_min - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Convert combined image back to PIL format and save
combined_image_pil = Image.fromarray(combined_image)
combined_image_pil.save(output_path_combined, format="JPEG", quality=95)

# Display the combined result
plt.figure(figsize=(12, 12))  # Increase figure size for better visibility
#plt.imshow(combined_image)
#plt.axis("off")
#plt.title("Combined House Detection, Tree Segmentation, and Building Segmentation")
#
plt.show()
print(f"Combined result saved at {output_path_combined}")

# Download the combined image to your local machine
files.download(output_path_combined)

Plot Metrics

# Evaluate Custom YOLOv8 Detector Performance


Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov8s_results_80`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# Start tensorboard
# Launch after you have started training
%load_ext tensorboard
%tensorboard --logdir runs